<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/Leonardo_Pacheco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Leonardo Augusto da Silva Pacheco'
print(f'Meu nome é {nome}.')

Meu nome é Leonardo Augusto da Silva Pacheco.


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.1 MB/s 
     |████████████████████████████████| 596 kB 53.8 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 69.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import tqdm
from tqdm.notebook import tqdm

In [4]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  8 16:03:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List
from torch import LongTensor

def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids

class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.
        self.max_seq_length = max_seq_length
        self.padding = [tokenizer.pad_token_id] * max_seq_length

        inputs = []
        targets = []
    
        for text in tqdm(texts):
            tokens = tokenize('[CLS] ' + text, tokenizer)
            sentences = len(tokens) // max_seq_length
            for i in range(sentences + 1):
                first = i * max_seq_length
                x = tokens[first : first + max_seq_length]
                y = tokens[first + 1:first + max_seq_length + 1]
                if len(x) < max_seq_length:
                    x += self.padding[:max_seq_length - len(x)]
                if len(y) < max_seq_length:
                    y += self.padding[:max_seq_length - len(y)]
                inputs.append(x)
                targets.append(y)
        
        self.inputs  = LongTensor(inputs)
        self.targets = LongTensor(targets)
        

    def __len__(self):
        # Escreva aqui seu código.
        return len(self.inputs)

    def __getitem__(self, idx):
        # Escreva aqui seu código.
        return self.inputs[idx], self.targets[idx]

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


In [8]:
dummy_texts = ['Eu gosto de correr mas estou cansado', 'Ela gosta muito de comer pizza mas está de dieta agora']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=100, shuffle=False)
print(len(dummy_dataset))
for x, y in dummy_loader:
    print('Next batch:')
    print(x)
    print(y)

  0%|          | 0/2 [00:00<?, ?it/s]

4
Next batch:
tensor([[  101,  3396, 10303,   125, 13239,   449, 12044,   822, 13550],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,   449],
        [  698,   125, 14559,  2535,     0,     0,     0,     0,     0]])
tensor([[ 3396, 10303,   125, 13239,   449, 12044,   822, 13550,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0],
        [ 1660,  5971,   785,   125,  1847, 13779, 15616,   449,   698],
        [  125, 14559,  2535,     0,     0,     0,     0,     0,     0]])


## Inicialização do Netune

In [10]:
!pip install -U neptune-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import neptune.new as neptune

run = neptune.init(
    project="leonardo3108/IA025AULA9",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhZDJhYWRmZi0zZmE0LTRhYzAtYThlMS1iYmJjMzU1NWU5YzQifQ==",
)  # your credentials

https://app.neptune.ai/leonardo3108/IA025AULA9/e/IA025AULA9-4
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [13]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-08 16:04:43--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   127MB/s    in 9.2s    

2022-06-08 16:04:52 (128 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [14]:
# Load datasets
max_seq_length = 9

train_examples = 180000
valid_examples =  20000
test_examples  =  50000

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 250000 lines.


  0%|          | 0/180000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [15]:
training_texts = valid_texts = test_texts = texts = None

In [16]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 22475700
valid examples: 2519650
test examples: 6241258


In [17]:
from torch.nn import Module, Linear, Softmax

class MultiHeadAttentionBlock(Module):
    def __init__(self, num_heads: int, max_seq_length: int, dim: int):
        super(MultiHeadAttentionBlock, self).__init__()

        self.L = max_seq_length
        self.D = dim
        self.H = num_heads
        self.D_H = self.D // self.H

        self.causal_mask = torch.tril(torch.ones(self.L, self.L)).bool()

        self.Wq = Linear(self.D, self.D, bias=False)
        self.Wk = Linear(self.D, self.D, bias=False)
        self.Wv = Linear(self.D, self.D, bias=False)

        self.softmax = Softmax(dim=-1)

    def forward(self, X: torch.LongTensor, attn_mask: torch.BoolTensor):
        causal_mask = self.causal_mask.to(X.device)

        Q = self.Wq(X).reshape(-1, self.L, self.H, self.D_H)
        K = self.Wk(X).reshape(-1, self.L, self.H, self.D_H)
        V = self.Wv(X).reshape(-1, self.L, self.H, self.D_H)
        Q, K, V = Q.transpose(1, 2), K.transpose(1, 2), V.transpose(1, 2)

        scores = Q @ torch.transpose(K, 2, 3) / math.sqrt(self.D)
        scores = scores.masked_fill(~causal_mask | ~attn_mask, -1e9)
        probs = self.softmax(scores)

        E = probs @ V
        E = E.transpose(1, 2).contiguous().reshape(-1, self.L, self.D)
        return E

In [18]:
from torch.nn import Sequential

class SequenceWithParams(Sequential):
    def forward(self, input, *args, **kwargs):
        for module in self:
            input = module(input, *args, **kwargs)
        return input

In [19]:
from torch.nn import Embedding, LayerNorm, ReLU

class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, max_seq_length, dim, n_layers, pad_token_id):
        super(LanguageModel, self).__init__()

        self.V = vocab_size
        self.L = max_seq_length
        self.D = dim
        self.pad_token_id = pad_token_id
        hidden_size = 512
        num_heads = 8

        self._positions = torch.arange(self.L, dtype=torch.long).unsqueeze(0)
        self.C = Embedding(self.V, self.D, padding_idx=self.pad_token_id)
        self.P = Embedding(self.L, self.D, padding_idx=self.pad_token_id)

        self.attention = SequenceWithParams(*[MultiHeadAttentionBlock(num_heads, self.L, self.D) for _ in range(n_layers)])
        self.Wo = Linear(self.D, self.D, bias=False)

        self.dense = Sequential(
            LayerNorm(self.D),
            Linear(self.D, hidden_size),
            ReLU(),
            LayerNorm(hidden_size),
            Linear(hidden_size, hidden_size),
            ReLU(),
            Linear(hidden_size, self.V, bias=False)
        )

    def forward(self, inputs: torch.LongTensor):
        positions = self._positions.repeat(inputs.shape[0], 1).to(inputs.device)
        attn_mask = (inputs != self.pad_token_id)[:, None, None, :]

        X = self.C(inputs) + self.P(positions)
        E = self.attention(X, attn_mask)
        E = self.Wo(E)
        logits = self.dense(torch.squeeze(E + X, dim=1))

        return logits

## Teste o modelo com um exemplo

In [20]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = torch.stack([sample_input, sample_input]).squeeze(1)
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([2, 9])
sample_output.shape: torch.Size([2, 9, 29794])


In [21]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 17487680


## Assert da Perplexidade


In [22]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30264
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [23]:
max_examples = 50_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=128,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=512, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=512)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        run['train/batch_loss'].log(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))
            run['train/perplexity'].log(train_ppl)

            with torch.no_grad():
                valid_losses = []
                for input, target in validation_loader:
                    valid_loss = validation_step(input.to(device), target.to(device))
                    valid_losses.append(valid_loss)
                    run['valid/batch_loss'].log(valid_loss)
                valid_ppl = np.exp(np.average(valid_losses))
                run['valid/perplexity'].log(valid_ppl)

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

      
        n_examples += len(train_input_ids)
        step += 1
         
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30187.08, valid ppl: 28686.79
10000 steps; 5120000 examples so far; train ppl: 293.80, valid ppl: 213.39
20000 steps; 10240000 examples so far; train ppl: 192.00, valid ppl: 184.07
30000 steps; 15360000 examples so far; train ppl: 172.14, valid ppl: 170.21
40000 steps; 20480000 examples so far; train ppl: 161.23, valid ppl: 161.44
50000 steps; 25600000 examples so far; train ppl: 152.17, valid ppl: 155.91
60000 steps; 30720000 examples so far; train ppl: 147.03, valid ppl: 151.59
70000 steps; 35840000 examples so far; train ppl: 143.68, valid ppl: 147.98
80000 steps; 40960000 examples so far; train ppl: 140.77, valid ppl: 145.06
90000 steps; 46080000 examples so far; train ppl: 137.56, valid ppl: 142.75


## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [24]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 139.7781352192604


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [25]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz muito
Eu gosto de comer pizza pois me faz muito bem
Eu gosto de comer pizza pois me faz muito bem,
Eu gosto de comer pizza pois me faz muito bem, mas
Eu gosto de comer pizza pois me faz muito bem, mas não
Eu gosto de comer pizza pois me faz muito bem, mas não é
Eu gosto de comer pizza pois me faz muito bem, mas não é o
Eu gosto de comer pizza pois me faz muito bem, mas não é o que
Eu gosto de comer pizza pois me faz muito bem, mas não é o que é
Eu gosto de comer pizza pois me faz muito bem, mas não é o que é o
Eu gosto de comer pizza pois me faz muito bem, mas não é o que é o que
Eu gosto de comer pizza pois me faz muito bem, mas não é o que é o que é
Eu gosto de comer pizza pois me faz muito bem, mas não é o que é o que é o
Eu gosto de comer pizza pois me faz muito bem, mas não é o que é o que é o que
Eu gosto de comer pizza pois me faz muito bem, mas não é o que é o que é o que é
Eu gosto de comer pizza pois me faz muito bem, mas não é o que é o qu

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.